In [1]:
import random
import dataparser
from ARMA import ARMA
import numpy as np
from sklearn.model_selection import KFold, ShuffleSplit
from tqdm import tqdm

In [2]:
rootdir = '/Volumes/My Passport/AI_Research/data/physionet.org/files/chbmit/1.0.0/'
patient_id = 'chb01'
parser = dataparser.DataParser(rootdir, patient_id)

In [3]:
ictal_filenames = parser.seizure_filenames()
preictal_filenames = [x for x in ictal_filenames if parser.check_preictal_interval_exists(x, parser.mins_in_secs(15))]
interictal_filenames = parser.no_seizure_filenames()
preictal_labels = np.ones((len(preictal_filenames)))
interictal_labels = -1 * np.ones((len(interictal_filenames)))

In [21]:
# Cross Validation and Shuffle Split
X_preictal = np.array(preictal_filenames)
y_preictal = np.array(preictal_labels)

X_interictal = np.array(interictal_filenames)
y_interictal = np.array(interictal_labels)

kf = KFold(n_splits=6)
kf.get_n_splits(X_preictal)
print(kf)

rs = ShuffleSplit(n_splits=len(interictal_filenames), test_size=.01, random_state=0)
rs.get_n_splits(X_interictal)
print(rs)

print('-')

for interictal_train_index, interictal_test_index in rs.split(X_interictal, y_interictal):
    print("Interictal TRAIN:", interictal_train_index, "Interictal TEST:", interictal_test_index)
    print("---")
    # load interictal test data
    interictal_test_filename = X_interictal[interictal_test_index].item()
    interictal_test_filepath = rootdir + patient_id + '/' + interictal_test_filename
    interictal_test_data = parser.data_all(interictal_test_filepath)
    print('interictal test data:', interictal_test_data.shape)
    # load interictal train data
    interictal_train_filenames = [X_interictal[i] for i in interictal_train_index]
    interictal_train_filepaths = [rootdir + patient_id + '/' + x for x in interictal_train_filenames]
    interictal_train_data = []
    for i in tqdm(range(len(interictal_train_filepaths)), desc='Load interictal train data'):
        data = parser.data_all(interictal_train_filepaths[i])
        interictal_train_data.append(data)
    
    for preictal_train_index, preictal_test_index in kf.split(X_preictal, y_preictal):
        print("Preictal TRAIN:", preictal_train_index, "Preictal TEST:", preictal_test_index)
        print("---")
        # load preictal test data
        preictal_test_filename = X_preictal[preictal_test_index].item()
        preictal_test_filepath = rootdir + patient_id + '/' + preictal_test_filename
        preictal_test_data = parser.data_all(preictal_test_filepath)
        print('preictal test data:', preictal_test_data.shape)
        # load preictal train data
        preictal_train_filenames = [X_preictal[i] for i in preictal_train_index]
        preictal_train_filepaths = [rootdir + patient_id + '/' + x for x in preictal_train_filenames]
        preictal_train_data = []
        for i in range(len(preictal_train_filepaths)):
            is_picked = parser.check_preictal_interval_exists(preictal_train_filenames, parser.mins_in_secs(15))
            if is_picked:
                start, end = parser.preictal_interval_times(preictal_train_filepaths[i], parser.mins_in_secs(15))
                data = parser.data_interval(preictal_train_filepaths[i], start, end)
                preictal_train_data.append(data)

KFold(n_splits=6, random_state=None, shuffle=False)
ShuffleSplit(n_splits=35, random_state=0, test_size=0.01, train_size=None)
-
Interictal TRAIN: [20 16 28 22 15 10  2 11 27 25 34 32 26 30  8 13  5 17 14 31 24  1 12  6
 23  4 18 21 19  9  7 33  3  0] Interictal TEST: [29]
---


Load interictal train data:   0%|          | 0/34 [00:00<?, ?it/s]

interictal test data: (23, 921600)


Load interictal train data: 100%|██████████| 34/34 [00:26<00:00,  1.30it/s]


Preictal TRAIN: [1 2 3 4 5] Preictal TEST: [0]
---
preictal test data: (23, 921600)
Preictal TRAIN: [0 2 3 4 5] Preictal TEST: [1]
---
preictal test data: (23, 921600)
Preictal TRAIN: [0 1 3 4 5] Preictal TEST: [2]
---
preictal test data: (23, 921600)
Preictal TRAIN: [0 1 2 4 5] Preictal TEST: [3]
---
preictal test data: (23, 921600)
Preictal TRAIN: [0 1 2 3 5] Preictal TEST: [4]
---
preictal test data: (23, 921600)
Preictal TRAIN: [0 1 2 3 4] Preictal TEST: [5]
---
preictal test data: (23, 595200)
Interictal TRAIN: [34 12 21 24 16  7 29 13 15 27 30 33 31 22  8  6  5  4 26 28  2 18 11  3
 23 20 10  0 25  9  1 32 14 19] Interictal TEST: [17]
---


...

interictal test data: (23, 921600)


Load interictal train data: 100%|██████████| 34/34 [00:28<00:00,  1.20it/s]


Preictal TRAIN: [1 2 3 4 5] Preictal TEST: [0]
---
preictal test data: (23, 921600)
Preictal TRAIN: [0 2 3 4 5] Preictal TEST: [1]
---
preictal test data: (23, 921600)
Preictal TRAIN: [0 1 3 4 5] Preictal TEST: [2]
---
preictal test data: (23, 921600)
Preictal TRAIN: [0 1 2 4 5] Preictal TEST: [3]
---
preictal test data: (23, 921600)
Preictal TRAIN: [0 1 2 3 5] Preictal TEST: [4]
---
preictal test data: (23, 921600)
Preictal TRAIN: [0 1 2 3 4] Preictal TEST: [5]
---
preictal test data: (23, 595200)
Interictal TRAIN: [22 18 23 20  1  8 29 14 25 30 19 24 27 13 31 11  6  5 26 17  4 28 16  7
  9  3 12 33  0 32  2 21 10 15] Interictal TEST: [34]
---
interictal test data: (23, 921600)


Load interictal train data: 100%|██████████| 34/34 [00:28<00:00,  1.17it/s]


Preictal TRAIN: [1 2 3 4 5] Preictal TEST: [0]
---
preictal test data: (23, 921600)
Preictal TRAIN: [0 2 3 4 5] Preictal TEST: [1]
---
preictal test data: (23, 921600)
Preictal TRAIN: [0 1 3 4 5] Preictal TEST: [2]
---
preictal test data: (23, 921600)
Preictal TRAIN: [0 1 2 4 5] Preictal TEST: [3]
---
preictal test data: (23, 921600)
Preictal TRAIN: [0 1 2 3 5] Preictal TEST: [4]
---
preictal test data: (23, 921600)
Preictal TRAIN: [0 1 2 3 4] Preictal TEST: [5]
---
preictal test data: (23, 595200)


In [15]:
# ARMA
seed = 42
fs = 256 # sampling frequency in Hz
N = 500
p = 2
m = 5
sig = 

np.random.seed(seed)

fp = fs / N                                           # Prediction frequency
t_s = 1 / fs                                               # Input signal time period

a_k_list = []                                              # Sequential buffer for  MA samples
a_h_k_list = []                                            # Sequential buffer for AR signal samples
k_list = []                                                # Sequential buffer for time index in prediction point _k

n_c = sig.shape[0]                                         # Number of input channels
n = sig.shape[1]                                           # Total number of input samples
a = a_scale * np.random.randn(n_c, n_i)               # Initialise AR coefficients
c = m * np.ones(m)                               # Initialise MA coefficients
c = c / c.sum()

Ik = N                                                # Window width
for _k in tqdm(range(Ik + n_i, n)):                         # Sliding window
    if (_k % N == 0):                                 # Decimation policy: _k occurs once every N samples
        w_start = _k - Ik - n_i + 1                   # Starting index of sliding window (end index is maintained by _k)
        a_h = np.zeros((n_c, n_i))                    # AR parameter estimates from samples in window
        for _i in range(n_c):                              # Iterate channels
            x_t = sig[_i, w_start:_k]                       # Multi-channel window over input signal
            N_w = len(x_t)
            ymat = np.zeros((N_w - n_i, n_i))
            yb = np.zeros((N_w - n_i, n_i))
            for _c in range(n_i, 0, -1):                   # Past dependency of AR up to model order
                ymat[ : , n_i - _c] = x_t[n_i - _c : -_c]
            yb = x_t[n_i:]
            a_h[_i] = np.linalg.pinv(ymat) @ yb            # Least squares solution to optimal parameters via Moore-Penrose Pseudo Inverse
        a_k = np.zeros((n_c, n_i))
        a_h_k_idx = len(a_h_k_list) - 1                    # Index to most recent block of AR parameters of shape: (n_c, n_i)
        for _j in range(m):                                # MA smoothing of AR parameters going back m units of time, in timescale k
            if len(a_h_k_list) > m:                        # Only begin smoothing once unit of time elapsed is greater than m
                a_k = c[_j] * a_h_k_list[a_h_k_idx - _j]
        k_list.append(_k)
        a_h_k_list.append(a_h)
        a_k_list.append(a_k)
k = np.array(k_list)
a_h_k = np.array(a_h_k_list)
a_k = np.array(a_k_list)

NameError: name 'sig' is not defined